In [1]:
import csv
from dataclasses import dataclass

@dataclass
class LogEntry:
  time: float
  source: str

# TODO: token parsing
@dataclass
class ATCommand(LogEntry):
  command: str

@dataclass
class BulkData(LogEntry):
  data: bytes
  end: bool


def parse_log_entry(row):
  time = float(row['Time'])
  source = 'host' if row['Source'] == 'host' else 'device'
  data_length = int(row['Data length [bytes]'])
  leftover_data = row['Leftover Capture Data']
  at_stream = row['AT Stream'].replace('\\\\r', '\r')
  if at_stream:
    if data_length != len(at_stream):
      raise ValueError('Data length does not match AT Stream length', at_stream)
    if not at_stream.endswith('\r'):
      raise ValueError('AT Stream does not end with return character')
    if not at_stream.startswith('AT'):
      raise ValueError('AT Stream does not start with "AT"')
    return ATCommand(time, source, at_stream[2:-1])
  elif leftover_data:
    data = bytes.fromhex(leftover_data)
    if data_length != len(data):
      raise ValueError('Data length does not match Leftover Capture Data length')
    end = False
    if data[-1] == 0x3e:
      data = data[:-1]
      end = True
    elif source == 'host':
      end = True
    return BulkData(time, source, data, end)
  else:
    raise ValueError('No data in log entry')


def parse_log(filename):
  with open(filename, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    data = None
    for row in reader:
      msg = parse_log_entry(row)
      if not isinstance(msg, BulkData):
        if data:
          data.end = True
          yield data
          data = None
        yield msg
        continue

      if data:
        if msg.source == data.source:
          data.data += msg.data
        else:
          data.end = True
          yield data
          data = msg
      else:
        data = msg

      if data.end:
        yield data
        data = None


In [2]:
def try_parse_uds(data):
  if len(data) < 2:
    return None

  try:
    data = bytes.fromhex(data)
  except:
    return None

  service_type = data[0]

  # 0x10 0x03 "DIAGNOSTIC_SESSION_CONTROL" "EXTENDED_DIAGNOSTIC"
  if service_type == 0x10:
    return f'DIAGNOSTIC_SESSION_CONTROL {hex(data[1])}'
  elif service_type == 0x22:
    return f'READ_DATA_BY_IDENTIFIER {data[1:].hex()}'
  # 0x27 0x03 "SECURITY_ACCESS" "REQUEST_SEED"
  # 0x27 0x04 "SECURITY_ACCESS" "SEND_KEY"
  elif service_type == 0x27:
    if data[1] in (0x01, 0x03):
      access_type = 'REQUEST_SEED'
    elif data[1] in (0x02, 0x04):
      access_type = 'SEND_KEY'
    else:
      access_type = 'UNKNOWN'
    return f'SECURITY_ACCESS {access_type} ({hex(data[1])})'

  return None

In [3]:
for m in parse_log('forscan_elm327_pscm_toggle_lca.csv'):
  direction = '>' if m.source == 'host' else '<'
  if isinstance(m, ATCommand):
    print(f'[{m.time:.3f}] {direction} AT{m.command}')
  elif isinstance(m, BulkData):
    res = ''
    for s in m.data.decode().split('\r'):
      if ':' in s:
        res += s[2:].strip() + ' '
      else:
        res += s.strip() + ' '

    uds = try_parse_uds(res.strip())

    print(f'[{m.time:.3f}] {direction} {res}')
    if uds:
      print(f'  {uds}')
    if m.source == 'device' and m.end:
      print()

[1.247] > ATRV
[1.262] < 14.4V   

[4.264] > ATRV
[4.279] < 14.4V   

[7.282] > ATRV
[7.297] < 14.4V   

[7.412] > ATSH000730
[7.422] < OK   

[7.426] > ATCRA738
[7.435] < OK   

[7.437] > 22DE00  
  READ_DATA_BY_IDENTIFIER de00
[7.451] < CAN ERROR   

[7.453] > 22DE00  
  READ_DATA_BY_IDENTIFIER de00
[7.480] < 01B 62DE00FF000A<RX ERROR 8C02000250003D 4D0DCD00000000 00000000000000  

[7.509] > s 
[7.514] < STOPPED   

[7.520] > 22DE00  
  READ_DATA_BY_IDENTIFIER de00
[7.543] < 01B 62DE00FF000A 8C02000250003D 4D0DCD00000000 00000000000000  

[7.587] > s 
[7.592] < STOPPED   

[7.611] > 22DE01  
  READ_DATA_BY_IDENTIFIER de01
[7.640] < 01B 62DE01FFFFFF FFFFFFFFFFFF00 FF000000000000 00000000000000  

[7.668] > s 
[7.673] < STOPPED   

[7.687] > 22DE02  
  READ_DATA_BY_IDENTIFIER de02
[7.703] < 01B 62DE02FFFF00 000200FF000000 00000000000000 00000000000000  

[7.743] > s 
[7.749] < STOPPED   

[7.762] > 22DE03  
  READ_DATA_BY_IDENTIFIER de03
[7.783] < 01B 62DE03000000 00000000000000 000000